In [2]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

In [3]:
data=pd.read_csv("/content/Churn_Modelling.csv")

In [4]:
df=pd.DataFrame(data)
df=df.drop(['RowNumber', 'CustomerId', 'Surname'],axis=1)

In [5]:
label_encoder=LabelEncoder()
df['Gender']=label_encoder.fit_transform(df['Gender'])
one_hot_encoder=OneHotEncoder(sparse_output=False)
df_geo=one_hot_encoder.fit_transform(df[["Geography"]])


In [6]:
df_geo=pd.DataFrame(df_geo,columns=["France","Germany","Spain"])

In [7]:
df=df.drop(["Geography"],axis=1)
df=pd.concat([df,df_geo],axis=1)

In [8]:
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,France,Germany,Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [9]:
X=df.drop(['Exited'],axis=1)
y=df['Exited']

In [10]:

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)


In [11]:
### Defining the model to use different parameter using kerasClassifier

def create_model(neuron=32,layer=1):
    model=Sequential()
    model.add(Dense(neuron,activation="relu",input_shape=(X_train.shape[1],)))

    for _ in range(layer-1):
      model.add(Dense(neuron,activation="relu"))
    model.add(Dense(1,activation="sigmoid"))
    model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

    return model


In [12]:
## create a Keras classifier
model=KerasClassifier(neuron=32,layer=1,build_fn=create_model,epochs=50,batch_size=10,verbose=0)

In [13]:
param_grid={
    'neuron':[16,32,64,128],
    'layer':[1,2,3],
    'batch_size':[10,32,64],
    'epochs':[10,50,100]
}

# Print package versions for debugging
import pkg_resources
print(f"scikeras version: {pkg_resources.get_distribution('scikeras').version}")
print(f"scikit-learn version: {pkg_resources.get_distribution('scikit-learn').version}")
print(f"keras version: {pkg_resources.get_distribution('keras').version}")

scikeras version: 0.13.0
scikit-learn version: 1.4.2
keras version: 3.10.0


/tmp/ipython-input-600884085.py:9: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


In [14]:
## Perform Grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=5)

In [ ]:
grid_result=grid.fit(X_train, y_train)

##Print the best parameters
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))